In [1]:
%%time
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
from shapely.ops import unary_union
import matplotlib.pyplot as plt
import geopandas as gpd
import numpy as np
import rasterio

bf = gpd.read_file('BFE.shp')

spacing = 0.00001
dist = np.arange(0, bf.length[0], spacing)
pts = [bf.boundary[0].interpolate(d) for d in dist] + [bf.boundary[0]]

multipoint = unary_union(pts)
pt_feature = gpd.GeoDataFrame(geometry=[multipoint], crs='EPSG:4326')

pt_feature = pt_feature.geometry.explode(index_parts=False)
pt_final = gpd.GeoDataFrame(geometry=pt_feature.iloc[:-2].geometry, crs='EPSG:4326')

rast = rasterio.open('dallas_1m.tif')
for p in pt_final['geometry']:
    x = p.xy[0][0]
    y = p.xy[1][0]
    r,c = rast.index(x,y)
    pt_final.loc[pt_final['geometry'] == p, 'elevation'] = rast.read(1)[r,c]

KeyboardInterrupt: 

In [5]:
elv = pt_final.loc[pt_final['elevation'].notnull()]
print('mean', elv['elevation'].mean())
print('std', elv['elevation'].std())

mean 133.8877599382791
std 8.872867672353848
